In [64]:
'''
Title:       Optimale Steuerung und Regelung:
Subttitle:   1. Aufgabe
Author:      Stefan Kaufmann
MaNr.        51867606
Date:        01.04.2023
'''

'\nTitle:       Optimale Steuerung und Regelung:\nSubttitle:   1. Aufgabe\nAuthor:      Stefan Kaufmann\nMaNr.        51867606\nDate:        01.04.2023\n'

# Optimale Steuerung und Regelung
## 1. Übung
### Stefan Kaufmann - 51867606

In [65]:
import numpy as np
import scipy as sp
import scipy.linalg as la
import matplotlib.pyplot as plt
import scipy.signal as sig
from scipy.optimize import minimize

mathematisches Pendel einer oszillierenden Punktmasse mit PID Regler    
$m \ddot{y}(t) + \omega^{2}y(t) = u(t)  \hspace{2cm} y(0) = 1, \hspace{1cm} \dot{y} = 0 \\
u(t) = K_{p}y(t) + K_{D}\dot{y}(t) + K_{I} \int_{0}^{t}y(\tau) d\tau 
$  
mit dem Kostenfunktional    
$
F(k,y(k;t)) = \frac{1}{2} \int_{0}^{\inf} (x^{T}(t)Qx(t) + ru^{2})dt
$


## a) Überführung in ein satisches Problem

$ x = 
\begin{bmatrix}
y & \dot{y} & \int_{0}^{t}y(\tau) d\tau 
\end{bmatrix} ^{T}
$

$
\dot{x} = 
\begin{bmatrix}
0 & 1 & 0  \\
\frac{-\omega^{2}}{m} & 0 & 0 \\
1 & 0 & 0
\end{bmatrix} x
+
\begin{bmatrix}
0 \\ \frac{1}{m} \\ 0
\end{bmatrix} u
$     
$
y = 
\begin{bmatrix}
1 & 0 & 0
\end{bmatrix} + 0u
$   
$
F(k,y(k;t)) = \frac{1}{2} \sum_{k=0}^{N} (x_{k}^{T}(t)Qx_{k}(t) + ru_{k}^{2})dt \\
mit \hspace{1cm}  u = kx_{k}  \hspace{1cm}  x_{k+1} = Ax_{k} + Bu_{k}  = (A+Bk)x_{k}
$

In [147]:
# Parameter
m  = 1
w  = 2
Q = la.block_diag(3,4,5)
R = 6

In [148]:
# State Space System
A = np.array([[0,       1, 0],
               [-w**2/m, 0, 0],
               [1,       0, 0]])

B = np.array([0,1/m,0])
B_ = np.array([[0,1/m,0]])

C = np.array([1,0,0])
D = 0


In [ ]:
global x0,N,nx 

N = 1000       # Anzahl von Zeitschritten N--> unendlich
nx = 3                              # Anzahl von Zuständen
x0 = np.array([1,0,0])       # Startzustand --> zu stabilisiern
dt = 0.01

Ad,Bd,Cd,Dd,Ta = sig.cont2discrete((A,B_.T,C,D),dt, method='foh')  # Discretisierung damit dt --> gegen 0 geht  (= analytische Lösung)
def rollout(k,x):
    u     = k@x    
    dx    = Ad@x + Bd.T*u    
    #dx    =A@x +B*u
    return dx , u

def cost(k):   
       
    x_new = np.zeros((nx,N))
    u = np.zeros((1,N))
    
    x_new[:,1], u[:,0] = rollout(k,x0)  

    cost_ = x0@Q@x0*0 + R*u[:,0]**2*0
   
    for i in range(1,N-2): 
        # laufende Kosten
        x_new[:,i+1], u[:,i] = rollout(k,x_new[:,i])       
        
        cost_ += x_new[:,i+1]@Q@x_new[:,i+1]
        cost_ += u[:,i]**2*R
   
    return cost_   

print(cost([-1,-1,-1]))   



[2696.54608729]


In [136]:
# Lösung der algebraischen Ricatti Gleichung
P = la.solve_continuous_are(A,B_.T,Q,R)
K = -B@P/R
print('analytische Lösung',K)
Pd = la.solve_discrete_are(Ad,Bd,Q,R)
Kd = -Bd.T@Pd/R
print('zeitdiskrete Lösung',Kd)

analytische Lösung [-0.30926715 -1.13366704 -0.91287093]
zeitdiskrete Lösung [[-0.31306553 -1.13399859 -0.91338852]]


In [167]:
# Kontrolle der Kostenfunktion
k0 = np.array([-2, -2, -2])
res = minimize(cost, k0, method="nelder-mead", options={"disp": True})
print(res)
#print(cost(res.x))

Optimization terminated successfully.
         Current function value: 2386.143945
         Iterations: 85
         Function evaluations: 157
 final_simplex: (array([[-0.31450845, -1.13095113, -0.91569571],
       [-0.31450521, -1.13095741, -0.91576243],
       [-0.31449919, -1.13092152, -0.91564481],
       [-0.31446472, -1.13092549, -0.91565748]]), array([2386.14394454, 2386.1439447 , 2386.14394476, 2386.14394497]))
           fun: 2386.1439445405163
       message: 'Optimization terminated successfully.'
          nfev: 157
           nit: 85
        status: 0
       success: True
             x: array([-0.31450845, -1.13095113, -0.91569571])


## b) Gradient des Kostenfunktionals


In [195]:
def grad(k):    
    gradient = x0@k*x0*R
    xnew,u   = rollout(k,x0)

    for i in range(1,N): 
        xnew,u  = rollout(k,xnew.flatten())                  
        
        gradient = np.add((xnew@Q).flatten(), gradient)
        
        gradient += xnew.flatten()*u*R
        xnew,u  = rollout(k,xnew.flatten())
    
    return gradient

In [186]:
from scipy.optimize import approx_fprime
def grad2(k):
    # Nummerisch Ableiten 
    return approx_fprime(k, cost, 1e-8)

In [196]:
print(grad([-0.314,-1,-0.9]))
print(grad([-2,-2,-2]))

[  67.98890513 -800.74260125  428.07253998]
[ -17.00526916 -795.38247478  398.7201746 ]


In [197]:
# Kontrolle des Gradienten
k0 = np.array([-2, -2, -2])
res = minimize(cost, k0, method="BFGS",jac=grad2, options={"disp": True})
print(res)

## c) Gradientenverfahren nach [Gra19, Abschnitt 3.3.2]